Now that we have a model and data it’s time to train, validate and test our model by optimizing its parameters on our data. Training a model is an iterative process; in each iteration the model makes a guess about the output, calculates the error in its guess (loss), collects the derivatives of the error with respect to its parameters (as we saw in the previous section), and optimizes these parameters using gradient descent. For a more detailed walkthrough of this process, check out this video on backpropagation from 3Blue1Brown.

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor


In [2]:
training_data =  datasets.FashionMNIST(
    root = "data",
    train  =  True,
    download = True,
    transform = ToTensor()
)

testing_data =  datasets.FashionMNIST(
    root = "data",
    train = False,
    download = True,
    transform =  ToTensor()
)

train =  DataLoader(training_data, batch_size=64)
test =  DataLoader(testing_data, batch_size=64)


100%|██████████| 26421880/26421880 [00:03<00:00, 8646334.87it/s] 


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 138511.29it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 2533669.32it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 19296047.36it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [5]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten =  nn.Flatten()
    self.linear_relu_stack =  nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512,512),
        nn.ReLU(),
        nn.Linear(512,10),
    )
  def forward(self,x):
    x =  self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

model=  NeuralNetwork()


# Hyperparameters
Hyperparameters are adjustable parameters that let you control the model optimization process. Different hyperparameter values can impact model training and convergence rates (read more about hyperparameter tuning)

We define the following hyperparameters for training:
1. Number of Epochs - the number times to iterate over the dataset

2. Batch Size - the number of data samples propagated through the network before the parameters are updated

3. Learning Rate - how much to update models parameters at each batch/epoch. Smaller values yield slow learning speed, while large values may result in unpredictable behavior during traini

In [6]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

## Optimization Loop
Once we set our hyperparameters, we can then train and optimize our model with an optimization loop. Each iteration of the optimization loop is called an epoch.

Each epoch consists of two main parts:
1. The Train Loop - iterate over the training dataset and try to converge to optimal parameters.

2. The Validation/Test Loop - iterate over the test dataset to check if model performance is improving.

Let’s briefly familiarize ourselves with some of the concepts used in the training loop. Jump ahead to see the Full Implementation of the optimization loop.

## Loss Function
When presented with some training data, our untrained network is likely not to give the correct answer. Loss function measures the degree of dissimilarity of obtained result to the target value, and it is the loss function that we want to minimize during training. To calculate the loss we make a prediction using the inputs of our given data sample and compare it against the true data label value.

Common loss functions include nn.MSELoss (Mean Square Error) for regression tasks, and nn.NLLLoss (Negative Log Likelihood) for classification. nn.CrossEntropyLoss combines nn.LogSoftmax and nn.NLLLoss.

We pass our model’s output logits to nn.CrossEntropyLoss, which will normalize the logits and compute the prediction error.

In [7]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

## Optimizer
Optimization is the process of adjusting model parameters to reduce model error in each training step. Optimization algorithms define how this process is performed (in this example we use Stochastic Gradient Descent). All optimization logic is encapsulated in the optimizer object. Here, we use the SGD optimizer; additionally, there are many different optimizers available in PyTorch such as ADAM and RMSProp, that work better for different kinds of models and data.

We initialize the optimizer by registering the model’s parameters that need to be trained, and passing in the learning rate hyperparameter.

In [8]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

Inside the training loop, optimization happens in three steps:
1. Call optimizer.zero_grad() to reset the gradients of model parameters. Gradients by default add up; to prevent double-counting, we explicitly zero them at each iteration.

2. Backpropagate the prediction loss with a call to loss.backward(). PyTorch deposits the gradients of the loss w.r.t. each parameter.

3. Once we have our gradients, we call optimizer.step() to adjust the parameters by the gradients collected in the backward pass.


## Full Implementation
We define train_loop that loops over our optimization code, and test_loop that evaluates the model’s performance against our test data

In [12]:
def train_loop(dataloader, model, loss_fn, optimizer):
  size =  len(dataloader.dataset)
  # Set the model to training mode - important for batch normalization and dropout layers
  # Unnecessary in this situation but added for best practices
  model.train()
  for batch,(X, y) in enumerate(dataloader):
    pred = model(X)
    loss =  loss_fn(pred,y)
    #backpropogation
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if batch % 100 == 0:
      loss, current =  loss.item(), (batch+1)*len(X)
      print(f"loss:{loss:>7f}[{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size =  len(dataloader.dataset)
    num_batches =  len(dataloader)
    test_loss , correct = 0,0
    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True

    with torch.no_grad():
      for X,y in dataloader:
        pred =  model(X)
        test_loss += loss_fn(pred,y).item()
        correct += (pred.argmax(1)==y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

We initialize the loss function and optimizer, and pass it to train_loop and test_loop. Feel free to increase the number of epochs to track the model’s improving performance.

In [13]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train, model, loss_fn, optimizer)
    test_loop(test, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss:2.317640[   64/60000]
loss:2.300056[ 6464/60000]
loss:2.283798[12864/60000]
loss:2.266202[19264/60000]
loss:2.241877[25664/60000]
loss:2.227216[32064/60000]
loss:2.229166[38464/60000]
loss:2.207797[44864/60000]
loss:2.201771[51264/60000]
loss:2.150528[57664/60000]
Test Error: 
 Accuracy: 40.7%, Avg loss: 2.156500 

Epoch 2
-------------------------------
loss:2.176528[   64/60000]
loss:2.164522[ 6464/60000]
loss:2.117493[12864/60000]
loss:2.119777[19264/60000]
loss:2.066458[25664/60000]
loss:2.023382[32064/60000]
loss:2.036442[38464/60000]
loss:1.976722[44864/60000]
loss:1.977183[51264/60000]
loss:1.888890[57664/60000]
Test Error: 
 Accuracy: 51.4%, Avg loss: 1.898631 

Epoch 3
-------------------------------
loss:1.940483[   64/60000]
loss:1.911478[ 6464/60000]
loss:1.810316[12864/60000]
loss:1.828890[19264/60000]
loss:1.727642[25664/60000]
loss:1.687638[32064/60000]
loss:1.687206[38464/60000]
loss:1.609777[44864/60000]
loss:1.627018[51264/